# Overdetermined systems

A system of polynomial equations $f=(f_1(x_1,\ldots, x_m),\ldots,  f_n(x_1,\ldots,x_m))$ is called *overdetermined*, if it has more equations than variables; i.e., when $n>m$. HomotopyContinuation.jl can solve overdetermined systems. Here is a simple example.

$$f(x,y,z) = \begin{bmatrix} xz-y^2 \\\ y-z^2 \\\ x-yz \\\ x + y + z + 1\end{bmatrix}.$$

This system has 4 equation in 3 variables. One might expect that it has no solution, but actually it has solutions, as is explained [here](https://en.wikipedia.org/wiki/Rational_normal_curve).

The Julia code is as follows

In [ ]:
using HomotopyContinuation
@polyvar x y z
f = [
    x*z - y^2, 
    y - z^2, 
    x - y*z, 
    x + y + z + 1
]

solve(f)

Internally, HomotopyContinuation multiplies $f$ with a random $3\times 4$ matrix $A$ and then solves the quadratic system $Af$. For a generic $A$ the isolated zeros of $f$ are isolated zeros of $Af$. We can do this by hand:

In [ ]:
A = randn(ComplexF64, 3, 4)
S = solve(A*f)

and sort out the zeros of $f$ by hand by first evaluation $f$ at the entries of `S`

In [ ]:
f_values = map(s -> [fi([x, y, z] => s) for fi in f], solutions(S))

and then sorting out the zero entries

In [ ]:
zero_entries = findall(v -> maximum(abs.(v)) < 1e-10, f_values)
println("\n and the zeros are at $zero_entries")

However, evaluation of $f$ at the solutions of $Af$ is not a stable way of finding the zeros of $f$. This is ultimately because $f$ and $tf$ have the same zeros for all $t\in\mathbb{C}\backslash\{0\}$. HomotopyContinuation.jl checks which zeros of $Af$ converge quadratically fast towards zeros of $f$ in order to make a decision.

# Composite systems

If your polynomial system $f$ is given as the composition of systems it is more efficient to *not* expand $f$, but exploit this structure! 

For instance, if

$$ f = \begin{bmatrix} ab - 2\\  ac- 1\end{bmatrix}, \quad\text{and}\quad g =  \begin{bmatrix}x + y\\ y + 3\\ x + 2\end{bmatrix},$$

then you solve $f\circ g$ by

In [ ]:
using HomotopyContinuation
@polyvar a b c x y
f = [a * b - 2, a * c- 1]
g = [x + y, y + 3, x + 2]

solve(f ∘ g)

It is also possible to iterate the above process:

In [ ]:
@polyvar u v
h = [u^2 - 1, u + v - 2]
solve(f ∘ g ∘ h)